In [1]:
import torch,inspect
import argparse
from pathlib import Path
from functools import partial
import numpy as np
from argparse import Namespace
from dev import namespace_tools
# nested namespace arguement containing all elements associated to the training setup

notebook_run = Namespace(
    simple_hp = Namespace(
        batch_size= 32,
        d_model = 64,
        early_stop_thresh = np.inf, # default to np.inf
        nb_epochs = 1000,
        warm_up_epochs = 20,
    ),
    # parameters to limit the size of the dataset
    dset_truncation = Namespace(
        limit_length= 15,
        use_splitting = False,
        max_length_from_file = False,
    ),
    # parameters for the optimization algorithm
    opt_params = Namespace(
        unlinked_optimizer = partial(torch.optim.NAdam,lr=0.01),
        unlinked_scheduler = partial(torch.optim.lr_scheduler.ReduceLROnPlateau, mode='min', 
                                     factor=0.9, patience=20)
    ),
    # parameters to reload the model
    train_state_control = Namespace(             
        load_from_backup = True,
        restore_optimizer = True
    ),
    #paths from root
    paths = namespace_tools.Paths(
        path_dataset = "data/french_english_dataset/fra.txt",
        path_language_info = "models/language_info.pth",
        path_dataset_splitting = "dataset_splitting",
        path_model_and_dependencies = "models/sequence_translator_transformer_new.pth",
        root = "../.."
    )

)

In [2]:
notebook_run = namespace_tools.NameSpaceAggregation(notebook_run)
notebook_run.diffuse(globals())

In [3]:
from ploomber_engine.ipython import PloomberClient
from ploomber import DAG
from pathlib import Path
from ploomber.products import File

# initialize client
client = PloomberClient.from_path(Path("./training_setup.ipynb"),cwd=Path("../../"))
from argparse import Namespace

from translation_machine.models import transformer_mod
from translation_machine import sentence_mod

initial_namespace_as_dict = notebook_run.diffuse()
train_setup = client.get_namespace(initial_namespace_as_dict)
for key,val in train_setup.items():
        globals()[key] = val

Executing cell: 13: 100%|███████████████████████| 21/21 [00:02<00:00,  7.59it/s]


In [4]:
# revert to train mode
model.train()
model.training

True

In [5]:
from translation_machine import model_trainer_mod
model_trainer = model_trainer_mod.ModelTrainer(model,optimizer,train_data_loader,val_data_loader,baseline_loss)

In [6]:
from argparse import Namespace
from pathlib import Path
class Paths(Namespace):
    def __init__(self,**kwargs):
        assert "root" in kwargs,"root must be given with the paths"
        self.root = kwargs.pop("root")
        kwargs = {key:Path(self.root).joinpath(val) for key,val in kwargs.items()}
        for key,val in kwargs.items():
            assert val.exists()
        super(Paths,self).__init__(**kwargs)

paths = Paths(
    root = "../..",
    path_dataset = "data/french_english_dataset/fra.txt",
    path_language_info = "models/language_info.pth",
    path_dataset_splitting = "dataset_splitting",
    path_model_and_dependencies = "models/sequence_translator_transformer_new.pth"
)

In [7]:
## import matplotlib.pyplot as plt,numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
best_loss_val_mean = np.inf
best_epoch = scheduler.last_epoch

for epoch in tqdm(range(simple_hp.nb_epochs)):
    #import time
    #start = time.time()
    print(f"training for epoch {epoch}")
    print(f"for epoch {epoch} learning rate is {optimizer.param_groups[0]['lr']}" )
    print("training_step")
    loss_train,nb_words_per_batch_train,metric_train = model_trainer.train_on_epoch()
    print("validation_step")
    loss_val,nb_words_per_batch_val,metric_val = model_trainer.validate_on_epoch()

    sum_loss_train = torch.tensor(loss_train).sum()
    sum_loss_val = torch.tensor(loss_val).sum()
    mean_train_loss = sum_loss_train/sum(nb_words_per_batch_train)
    mean_val_loss = sum_loss_val/sum(nb_words_per_batch_val)

    scheduler.step(mean_val_loss)

        
    print(f"for epoch {epoch} mean loss on train {mean_train_loss}")
    print(f"for epoch {epoch} mean loss on val {mean_val_loss}")
        
    losses["train"].append(mean_train_loss)
    losses["val"].append(mean_val_loss)
    metrics["train"].append(metric_train)
    metrics["val"].append(metric_val)
    
    if (mean_val_loss < best_loss_val_mean):
        best_epoch = scheduler.last_epoch
        best_loss_val_mean = mean_val_loss

        model_training_state = {"model_params":model_trainer.model.state_dict(),
                               "model_inputs":model_inputs,
                              "optimizer":optimizer.state_dict(),
                              "scheduler":scheduler.state_dict(),
                              }
        results = { "losses":losses,
                   "metrics":metrics}
        new_back_up = dict()
        if "back_up" in globals():
            new_back_up["notebook_runs"] = back_up["notebook_runs"] + tuple([notebook_run.state_dict()])
        else:
            new_back_up["notebook_runs"] = tuple([notebook_run.state_dict()])

        new_back_up["results"] = results
        new_back_up["model_training_state"] = model_training_state
        
        back_up = new_back_up
        torch.save(back_up,paths.path_model_and_dependencies)
        print(f"saving for epoch {epoch}")
        
        plt.plot(losses["train"],"b*")
        plt.plot(losses["val"],"g*")
        plt.title("losses")
        plt.savefig("loss_curve")
        #import pdb;pdb.set_trace()
    elif epoch - best_epoch > simple_hp.early_stop_thresh  and epoch > simple_hp.warm_up_epochs:
        print("Early stopped training at epoch %d" % epoch)
        break  # terminate the training loop

    del loss_train,nb_words_per_batch_train,metric_train

    del loss_val,nb_words_per_batch_val,metric_val


  0%|                                                                            | 0/1000 [00:00<?, ?it/s]

training for epoch 0
for epoch 0 learning rate is 0.0025418658283290017
training_step
0 1.95886597266564
validation_step
0 1.9844744755671575
for epoch 0 mean loss on train 1.958866000175476
for epoch 0 mean loss on val 1.9844744205474854
saving for epoch 0


  0%|                                                                    | 1/1000 [00:01<26:27,  1.59s/it]

training for epoch 1
for epoch 1 learning rate is 0.0025418658283290017
training_step
0 2.068482178908128
validation_step
0 1.9683157113882213
for epoch 1 mean loss on train 2.0684821605682373
for epoch 1 mean loss on val 1.9683157205581665
saving for epoch 1


  0%|▏                                                                   | 2/1000 [00:02<15:07,  1.10it/s]

...

100%|█████████████████████████████████████████████████████████████████▊| 997/1000 [30:44<00:00,  9.76it/s]

training for epoch 997
for epoch 997 learning rate is 1.797010299914435e-05
training_step
0 2.0668665078970103
validation_step
0 1.9960645528940053
for epoch 997 mean loss on train 2.066866397857666
for epoch 997 mean loss on val 1.99606454372406
training for epoch 998
for epoch 998 learning rate is 1.797010299914435e-05
training_step
0 2.075634296123798
validation_step
0 2.0461813119741588
for epoch 998 mean loss on train 2.075634241104126
for epoch 998 mean loss on val 2.0461812019348145


100%|█████████████████████████████████████████████████████████████████▉| 999/1000 [30:44<00:00,  9.96it/s]

training for epoch 999
for epoch 999 learning rate is 1.797010299914435e-05
training_step
0 2.05783932025616
validation_step
0 2.0840624295748196
for epoch 999 mean loss on train 2.0578393936157227
for epoch 999 mean loss on val 2.084062337875366


100%|█████████████████████████████████████████████████████████████████| 1000/1000 [30:45<00:00,  1.85s/it]


In [8]:
for el in model_trainer.train_data_loader:
    print([el1.shape for el1 in el])

[torch.Size([15, 2]), torch.Size([15, 6]), torch.Size([15]), torch.Size([15])]


In [9]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.plot(results["losses"]["train"],"b*")
plt.plot(results["losses"]["val"],"g*")
plt.title("losses")
plt.savefig(f'test.png', bbox_inches='tight')